In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report

import random
import numpy as np
import torch

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If you are using CUDA
torch.backends.cudnn.deterministic = True  # For deterministic results
torch.backends.cudnn.benchmark = False  # For consistency across different environments

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMAGE_DIR = 'D:\\PAD-UFES\\images'  
METADATA_PATH = 'D:\\PAD-UFES\\metadata.csv'

metadata = pd.read_csv(METADATA_PATH)

def preprocess_metadata(metadata):
    metadata = metadata.fillna('UNK')

    boolean_cols = [
        'smoke',
        'drink',
        'pesticide',
        'skin_cancer_history',
        'cancer_history',
        'has_piped_water',
        'has_sewage_system',
        'itch',
        'grew',
        'hurt',
        'changed',
        'bleed',
        'elevation',
        'biopsed',
    ]
    # Ensure columns are strings and lowercase
    for col in boolean_cols:
        metadata[col] = metadata[col].astype(str).str.lower()
    
    # Map boolean columns to 1/0/-1
    boolean_mapping = {'true': 1, 'false': 0, 'unk': -1}
    for col in boolean_cols:
        metadata[col] = metadata[col].map(boolean_mapping)
    
    # Handle categorical variables
    categorical_cols = [
        'background_father',
        'background_mother',
        'gender',
        'region',
        'diagnostic',
    ]
    # Convert categorical columns to string
    for col in categorical_cols:
        metadata[col] = metadata[col].astype(str)
    
    # One-hot encode categorical variables
    metadata_encoded = pd.get_dummies(metadata[categorical_cols])
    
    # Normalize numerical variables
    numerical_cols = ['age', 'fitspatrick', 'diameter_1', 'diameter_2']
    # Ensure numerical columns are numeric
    for col in numerical_cols:
        metadata[col] = pd.to_numeric(metadata[col], errors='coerce')
    # Fill NaNs in numerical columns with the mean
    metadata[numerical_cols] = metadata[numerical_cols].fillna(metadata[numerical_cols].mean())
    # Scale numerical columns
    scaler = StandardScaler()
    metadata_numeric = metadata[numerical_cols]
    metadata_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(metadata_numeric), columns=numerical_cols
    )
    
    # Combine all metadata features
    metadata_processed = pd.concat(
        [metadata_numeric_scaled.reset_index(drop=True),
         metadata_encoded.reset_index(drop=True),
         metadata[boolean_cols].reset_index(drop=True)], axis=1
    )
    
    return metadata_processed

# Preprocess metadata
metadata_processed = preprocess_metadata(metadata)

def get_image_paths(metadata, image_dir):
    image_paths = []
    for idx, row in metadata.iterrows():
        filename = row['img_id']
        # Ensure filename is a string
        filename = str(filename)
        # Check if filename has an extension
        if not filename.endswith(('.jpg', '.jpeg', '.png')):
            # Try common extensions
            possible_extensions = ['.jpg', '.jpeg', '.png']
            found = False
            for ext in possible_extensions:
                filepath = os.path.join(image_dir, filename + ext)
                if os.path.isfile(filepath):
                    image_paths.append(filepath)
                    found = True
                    break
            if not found:
                print(f"Image file not found for ID: {filename}")
                image_paths.append(None)
        else:
            filepath = os.path.join(image_dir, filename)
            if os.path.isfile(filepath):
                image_paths.append(filepath)
            else:
                print(f"Image file not found: {filepath}")
                image_paths.append(None)
    metadata['ImagePath'] = image_paths
    return metadata

metadata = get_image_paths(metadata, IMAGE_DIR)

# Remove entries with missing images
metadata = metadata[metadata['ImagePath'].notnull()]
metadata_processed = metadata_processed.loc[metadata.index].reset_index(drop=True)
metadata = metadata.reset_index(drop=True)

# Drop diagnostic-related columns from features
diagnostic_cols = ['diagnostic_ACK', 'diagnostic_BCC', 'diagnostic_MEL', 'diagnostic_NEV', 'diagnostic_SCC', 'diagnostic_SEK']
metadata_processed = metadata_processed.drop(columns=diagnostic_cols)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(metadata['diagnostic'])
num_classes = len(label_encoder.classes_)

# Split data into features and labels
X_meta = metadata_processed.reset_index(drop=True)
X_img_paths = metadata['ImagePath'].reset_index(drop=True)
y = pd.Series(y_encoded)

X_train_meta, X_temp_meta, X_train_img_paths, X_temp_img_paths, y_train, y_temp = train_test_split(
    X_meta,
    X_img_paths,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_val_meta, X_test_meta, X_val_img_paths, X_test_img_paths, y_val, y_test = train_test_split(
    X_temp_meta,
    X_temp_img_paths,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

# Load augmented metadata + image paths
aug_meta_df   = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
aug_labels_df = pd.read_csv("D:/PAD-UFES/augmented_labels.csv")

# Combine augmented samples with training set
X_train_meta_final = pd.concat([X_train_meta, aug_meta_df], ignore_index=True)
X_train_img_paths_final = pd.concat([X_train_img_paths.reset_index(drop=True),
                                     aug_labels_df['ImagePath']], ignore_index=True)
y_train_final = pd.concat([y_train.reset_index(drop=True),
                           aug_labels_df['Label']], ignore_index=True)

class PADUFESDataset(Dataset):
    def __init__(self, img_paths, meta_data, labels, transform=None):
        self.img_paths = img_paths.reset_index(drop=True)
        self.meta_data = meta_data.reset_index(drop=True)
        self.labels = pd.Series(labels).reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        meta = torch.tensor(self.meta_data.iloc[idx].values.astype(np.float32))
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return image, meta, label

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    transforms.RandomRotation(70),          
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = PADUFESDataset(X_train_img_paths_final, X_train_meta_final, y_train_final, transform=train_transform)
val_dataset = PADUFESDataset(X_val_img_paths, X_val_meta, y_val, transform=val_test_transform)
test_dataset = PADUFESDataset(X_test_img_paths, X_test_meta, y_test, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"\n✅ Loaded Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

In [ ]:
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_accuracy_max = -np.Inf
        
    def __call__(self, val_acc, model):
        score = val_acc
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'Validation accuracy increased ({self.val_accuracy_max:.6f} --> {val_acc:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_accuracy_max = val_acc

from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def test(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            outputs = model(imgs, metas)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for images, meta, labels in pbar:
        images, meta, labels = images.to(device), meta.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, meta)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Optional: Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f'{running_loss/total:.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss/len(train_loader), correct/total

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, meta, labels in val_loader:
            images, meta, labels = images.to(device), meta.to(device), labels.to(device)
            
            outputs = model(images, meta)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss/len(val_loader), correct/total

def train_model_with_scheduler_and_checkpoint(
    model, train_loader, val_loader, optimizer, criterion, device, 
    epochs=20, patience=5, scheduler_patience=5, checkpoint_dir='checkpoints'):
    
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pt')
    
    early_stopping = EarlyStopping(
        patience=patience, 
        verbose=True, 
        path=checkpoint_path
    )
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='max',  # Changed to max since we're monitoring accuracy
        patience=scheduler_patience, 
        verbose=True,
        factor=0.1,
        min_lr=1e-6
    )
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'lr': []
    }
    
    best_model_epoch = None
    
    for epoch in range(epochs):
        print(f'\nEpoch {epoch+1}/{epochs}')
        
        # Training phase
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        
        # Validation phase
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Update scheduler based on validation accuracy
        scheduler.step(val_acc)
        
        # Early stopping check
        early_stopping(val_acc, model)
        if val_acc > early_stopping.val_accuracy_max:
            best_model_epoch = epoch + 1
            
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    # Load best model
    model.load_state_dict(torch.load(checkpoint_path))
    
    # Plot training curves
    plot_training_curves_with_checkpoint(history, best_model_epoch)
    
    return model, history

def plot_training_curves_with_checkpoint(history, best_model_epoch):
    epochs_range = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
    
    # Loss curves
    ax1.plot(epochs_range, history['train_loss'], label='Training Loss')
    ax1.plot(epochs_range, history['val_loss'], label='Validation Loss')
    if best_model_epoch:
        ax1.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    
    # Accuracy curves
    ax2.plot(epochs_range, history['train_acc'], label='Training Accuracy')
    ax2.plot(epochs_range, history['val_acc'], label='Validation Accuracy')
    if best_model_epoch:
        ax2.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.legend()
    
    # Learning rate curve
    ax3.plot(epochs_range, history['lr'], label='Learning Rate')
    if best_model_epoch:
        ax3.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_title('Learning Rate Schedule')
    ax3.set_yscale('log')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
import torch
import torch.nn as nn
import timm  
import torch.nn.functional as F

class EarlyFusionModel(nn.Module):
    def __init__(self, input_dim_meta, num_classes):
        super().__init__()
        
        # Embed metadata to smaller spatial dimensions first
        self.meta_embed = nn.Sequential(
            nn.Linear(input_dim_meta, 56 * 56),  # Smaller initial dimension
            nn.ReLU(),
            nn.BatchNorm1d(56 * 56),
            nn.Dropout(0.3)
        )
        
        # Modified Xception
        self.xception = timm.create_model('xception', pretrained=True)
        # Modify first conv layer to accept additional channel
        first_conv = self.xception.conv1
        self.xception.conv1 = nn.Conv2d(4, 32, kernel_size=3, stride=2, padding=1, bias=False)
        
        # Modify the final classification layer
        in_features = self.xception.fc.in_features  # Usually 2048 for Xception
        self.xception.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, num_classes)
        )
        
        # Initialize new channel weights
        with torch.no_grad():
            self.xception.conv1.weight.data[:, :3] = first_conv.weight.data
            # Initialize the new channel with smaller weights to prevent dominating
            self.xception.conv1.weight.data[:, 3:] = first_conv.weight.data.mean(dim=1, keepdim=True) * 0.1
        
    def forward(self, img, meta):
        # Reshape metadata to image-like format
        batch_size = img.shape[0]
        meta_reshaped = self.meta_embed(meta).view(batch_size, 1, 56, 56)
        
        # Upsample to match image dimensions
        meta_upsampled = F.interpolate(meta_reshaped, 
                                     size=(224, 224), 
                                     mode='bilinear', 
                                     align_corners=False)
        
        # Early fusion
        combined_input = torch.cat([img, meta_upsampled], dim=1)
        
        # Process through modified Xception
        out = self.xception(combined_input)
        return out
        
input_dim_meta = X_train_meta.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EarlyFusionModel(input_dim_meta, num_classes).to(device)

from torchinfo import summary
summary(model=model, 
        input_size=[(16, 3, 224, 224), (16, input_dim_meta)],  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)
xception_model = EarlyFusionModel(input_dim_meta, num_classes).to(device)
xception_model.load_state_dict(torch.load('D:\\PAD-UFES\\best_early_fusion_xceptionsmoteDA.pth'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xception_model.to(device)

true_labels, pred_labels = test(xception_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
print(input_dim_meta)

In [ ]:
import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_accuracy_max = -np.Inf
        
    def __call__(self, val_acc, model):
        score = val_acc
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
        elif score <= self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_acc, model):
        if self.verbose:
            print(f'Validation accuracy increased ({self.val_accuracy_max:.6f} --> {val_acc:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_accuracy_max = val_acc

from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def test(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            outputs = model(imgs, metas)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, desc='Training')
    for images, meta, labels in pbar:
        images, meta, labels = images.to(device), meta.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images, meta)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Optional: Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f'{running_loss/total:.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    return running_loss/len(train_loader), correct/total

def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, meta, labels in val_loader:
            images, meta, labels = images.to(device), meta.to(device), labels.to(device)
            
            outputs = model(images, meta)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss/len(val_loader), correct/total

def train_model_with_scheduler_and_checkpoint(
    model, train_loader, val_loader, optimizer, criterion, device, 
    epochs=20, patience=5, scheduler_patience=5, checkpoint_dir='checkpoints'):
    
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, 'best_model.pt')
    
    early_stopping = EarlyStopping(
        patience=patience, 
        verbose=True, 
        path=checkpoint_path
    )
    scheduler = ReduceLROnPlateau(
        optimizer, 
        mode='max',  # Changed to max since we're monitoring accuracy
        patience=scheduler_patience, 
        verbose=True,
        factor=0.1,
        min_lr=1e-6
    )
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'lr': []
    }
    
    best_model_epoch = None
    
    for epoch in range(epochs):
        print(f'\nEpoch {epoch+1}/{epochs}')
        
        # Training phase
        train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
        
        # Validation phase
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        # Update history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lr'].append(optimizer.param_groups[0]['lr'])
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
        
        # Update scheduler based on validation accuracy
        scheduler.step(val_acc)
        
        # Early stopping check
        early_stopping(val_acc, model)
        if val_acc > early_stopping.val_accuracy_max:
            best_model_epoch = epoch + 1
            
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
    
    # Load best model
    model.load_state_dict(torch.load(checkpoint_path))
    
    # Plot training curves
    plot_training_curves_with_checkpoint(history, best_model_epoch)
    
    return model, history

def plot_training_curves_with_checkpoint(history, best_model_epoch):
    epochs_range = range(1, len(history['train_loss']) + 1)
    
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
    
    # Loss curves
    ax1.plot(epochs_range, history['train_loss'], label='Training Loss')
    ax1.plot(epochs_range, history['val_loss'], label='Validation Loss')
    if best_model_epoch:
        ax1.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    
    # Accuracy curves
    ax2.plot(epochs_range, history['train_acc'], label='Training Accuracy')
    ax2.plot(epochs_range, history['val_acc'], label='Validation Accuracy')
    if best_model_epoch:
        ax2.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Training and Validation Accuracy')
    ax2.legend()
    
    # Learning rate curve
    ax3.plot(epochs_range, history['lr'], label='Learning Rate')
    if best_model_epoch:
        ax3.axvline(best_model_epoch, color='r', linestyle='--', label='Best Model')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_title('Learning Rate Schedule')
    ax3.set_yscale('log')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
import torch
import torch.nn as nn
import timm
import torch.nn.functional as F

class EarlyFusionModel(nn.Module):
    def __init__(self, input_dim_meta, num_classes):
        super().__init__()
        
        # Embed metadata to smaller spatial dimensions first
        self.meta_embed = nn.Sequential(
            nn.Linear(input_dim_meta, 56 * 56),  # Smaller initial dimension
            nn.ReLU(),
            nn.BatchNorm1d(56 * 56),
            nn.Dropout(0.3)
        )
        
        # Load PVT v2 model
        self.pvt = timm.create_model("pvt_v2_b1", pretrained=True, num_classes=num_classes)
        
        # Modify the first convolution layer to accept additional channel (4 instead of 3)
        first_conv = self.pvt.patch_embed.proj
        self.pvt.patch_embed.proj = nn.Conv2d(4, first_conv.out_channels, 
                                              kernel_size=first_conv.kernel_size,
                                              stride=first_conv.stride,
                                              padding=first_conv.padding,
                                              bias=first_conv.bias is not None)
        
        # Initialize new channel weights
        with torch.no_grad():
            self.pvt.patch_embed.proj.weight.data[:, :3] = first_conv.weight.data
            self.pvt.patch_embed.proj.weight.data[:, 3:] = first_conv.weight.data.mean(dim=1, keepdim=True) * 0.1

    def forward(self, img, meta):
        # Reshape metadata to image-like format
        batch_size = img.shape[0]
        meta_reshaped = self.meta_embed(meta).view(batch_size, 1, 56, 56)
        
        # Upsample to match image dimensions
        meta_upsampled = F.interpolate(meta_reshaped, 
                                       size=(224, 224), 
                                       mode='bilinear', 
                                       align_corners=False)
        
        # Early fusion
        combined_input = torch.cat([img, meta_upsampled], dim=1)
        
        # Process through modified PVT
        out = self.pvt(combined_input)
        return out

input_dim_meta = X_train_meta.shape[1]
num_classes = 6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EarlyFusionModel(input_dim_meta, num_classes).to(device)

from torchinfo import summary
summary(model=model, 
        input_size=[(16, 3, 224, 224), (16, input_dim_meta)],  
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

pv2_model = EarlyFusionModel(input_dim_meta=input_dim_meta, num_classes=num_classes).to(device)
pv2_model.load_state_dict(torch.load('D:\\PAD-UFES\\best_early_fusion_pvtv2smoteDA.pth'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pv2_model.to(device)

true_labels, pred_labels = test(pv2_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
import torch
import torch.nn as nn
import timm  
import torch.nn.functional as F

class EarlyFusionModel(nn.Module):
    def __init__(self, input_dim_meta, num_classes):
        super().__init__()
        
        # Embed metadata to smaller spatial dimensions first
        self.meta_embed = nn.Sequential(
            nn.Linear(input_dim_meta, 64 * 64),  # Updated for mobilevit's smaller receptive field
            nn.ReLU(),
            nn.BatchNorm1d(64 * 64),
            nn.Dropout(0.3)
        )
        
        # Load MobileViT model
        self.mobilevit = timm.create_model("mobilevit_s.cvnets_in1k", pretrained=True, num_classes=num_classes)
        
        # Inspect the model to identify the first conv layer
        # Modify the first conv layer to accept additional channel
        first_conv = self.mobilevit.stem.conv  # `stem.conv` is the correct initial layer
        self.mobilevit.stem.conv = nn.Conv2d(4, first_conv.out_channels, 
                                             kernel_size=first_conv.kernel_size, 
                                             stride=first_conv.stride, 
                                             padding=first_conv.padding, 
                                             bias=first_conv.bias)
        
        # Initialize new channel weights
        with torch.no_grad():
            self.mobilevit.stem.conv.weight.data[:, :3] = first_conv.weight.data
            # Initialize the new channel with smaller weights to prevent dominating
            self.mobilevit.stem.conv.weight.data[:, 3:] = first_conv.weight.data.mean(dim=1, keepdim=True) * 0.1

    def forward(self, img, meta):
        # Reshape metadata to image-like format
        batch_size = img.shape[0]
        meta_reshaped = self.meta_embed(meta).view(batch_size, 1, 64, 64)
        
        # Upsample to match image dimensions
        meta_upsampled = F.interpolate(meta_reshaped, 
                                       size=(224, 224),  # MobileViT expects 256x256
                                       mode='bilinear', 
                                       align_corners=False)
        
        # Early fusion
        combined_input = torch.cat([img, meta_upsampled], dim=1)
        
        # Process through modified MobileViT
        out = self.mobilevit(combined_input)
        return out

# Assuming X_train_meta and other variables are defined
input_dim_meta = X_train_meta.shape[1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EarlyFusionModel(input_dim_meta, num_classes).to(device)

from torchinfo import summary
summary(model=model, 
        input_size=[(16, 3, 224, 224), (16, input_dim_meta)],  # Updated for MobileViT input size
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

mobilevit_model = EarlyFusionModel(input_dim_meta=input_dim_meta, num_classes=num_classes).to(device)
mobilevit_model.load_state_dict(torch.load('D:\\PAD-UFES\\best_early_fusion_mobilevitDA.pth'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilevit_model.to(device)

true_labels, pred_labels = test(mobilevit_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
from transformers import AutoModelForImageClassification, AutoConfig
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForImageClassification, AutoImageProcessor

class EarlyFusionModel(nn.Module):
    def __init__(self, input_dim_meta, num_classes):
        super().__init__()
        
        # Embed metadata to smaller spatial dimensions
        self.meta_embed = nn.Sequential(
            nn.Linear(input_dim_meta, 56 * 56),
            nn.ReLU(),
            nn.BatchNorm1d(56 * 56),
            nn.Dropout(0.3)
        )
        
        # Load Swin Transformer from Hugging Face
        self.swin_transformer = AutoModelForImageClassification.from_pretrained(
            "microsoft/swin-tiny-patch4-window7-224",
            ignore_mismatched_sizes=True
        )
        
        # Replace the classifier to match the number of classes
        in_features = self.swin_transformer.classifier.in_features
        self.swin_transformer.classifier = nn.Linear(in_features, num_classes)
        
        # Access and modify the patch embedding layer
        patch_embed_layer = self.swin_transformer.swin.embeddings.patch_embeddings.projection
        self.swin_transformer.swin.embeddings.patch_embeddings.projection = nn.Conv2d(
            in_channels=4,  # Update to accept 4 input channels
            out_channels=patch_embed_layer.out_channels,
            kernel_size=patch_embed_layer.kernel_size,
            stride=patch_embed_layer.stride,
            padding=patch_embed_layer.padding,
            bias=patch_embed_layer.bias is not None,  # Ensure bias is a boolean
        )
        
        # Initialize weights for the new input channel
        with torch.no_grad():
            self.swin_transformer.swin.embeddings.patch_embeddings.projection.weight[:, :3] = patch_embed_layer.weight
            self.swin_transformer.swin.embeddings.patch_embeddings.projection.weight[:, 3:] = patch_embed_layer.weight.mean(dim=1, keepdim=True)

    def forward(self, img, meta):
        # Embed metadata
        batch_size = img.size(0)
        meta_reshaped = self.meta_embed(meta).view(batch_size, 1, 56, 56)
        
        # Upsample metadata to match image dimensions
        meta_upsampled = F.interpolate(meta_reshaped, size=(224, 224), mode="bilinear", align_corners=False)
        
        # Concatenate the image and metadata
        combined_input = torch.cat([img, meta_upsampled], dim=1)
        
        # Pass through Swin Transformer
        outputs = self.swin_transformer(pixel_values=combined_input)
        return outputs.logits


input_dim_meta = X_train_meta.shape[1]  # Number of metadata features
num_classes = 6  # Number of output classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EarlyFusionModel(input_dim_meta, num_classes).to(device)

# Model Summary
from torchinfo import summary
summary(
    model=model,
    input_size=[(16, 3, 224, 224), (16, input_dim_meta)],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

swin_model = EarlyFusionModel(input_dim_meta=input_dim_meta, num_classes=num_classes).to(device)
swin_model.load_state_dict(torch.load('D:\\PAD-UFES\\best_early_fusion_swintinysmoteDA.pth'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
swin_model.to(device)

true_labels, pred_labels = test(swin_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

<h1>Simple Averaging</h1>

In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

def test_ensemble(models, loader, device):
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            
            # Initialize a list to store individual model predictions
            model_outputs = []
            
            # Get predictions from each model and store
            for model in models:
                outputs = model(imgs, metas)
                model_outputs.append(outputs)
            
            # Stack model outputs along a new axis (axis 0: models)
            model_outputs = torch.stack(model_outputs, dim=0)
            
            # Average the outputs along the new axis (across models)
            avg_outputs = model_outputs.mean(dim=0)  # Averaging logits/probabilities

            # Convert the averaged outputs to predicted labels
            _, predicted = torch.max(avg_outputs, 1)

            # Store the predictions and the true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


# Function to calculate metrics and return mean ± std deviation
def calculate_metrics(true_labels, predictions):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


# Main loop to test the ensemble and calculate metrics with standard deviation
def evaluate_ensemble(models, test_loader, device, seeds):
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Loop through predefined seeds
    for seed in seeds:
        set_seed(seed)  # Set the random seed for reproducibility
        
        # Run the ensemble evaluation
        true_labels, ensemble_preds = test_ensemble(models, test_loader, device)

        # Calculate metrics for this run
        accuracy, precision, recall, f1 = calculate_metrics(true_labels, ensemble_preds)

        # Store metrics
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with 4 models:
models = [mobilevit_model, pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fi``xed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble(models, test_loader, device, seeds)


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class TeacherModel(nn.Module):
    def __init__(self, models, ensemble_method="mean"):
        """
        Teacher Model using Ensemble Learning.

        Args:
            models (list): List of trained models to use for ensembling.
            ensemble_method (str): "mean" for averaging logits, "vote" for majority voting.
        """
        super(TeacherModel, self).__init__()
        self.models = models
        self.ensemble_method = ensemble_method

        # Ensure all models are in eval mode and no gradients are computed
        for model in self.models:
            model.eval()
            for param in model.parameters():
                param.requires_grad = False

    def forward(self, img, meta):
        """
        Forward pass through the ensemble teacher model.

        Args:
            img (torch.Tensor): Batch of images.
            meta (torch.Tensor): Batch of metadata.

        Returns:
            torch.Tensor: The ensembled output (soft probabilities).
        """
        model_outputs = []

        with torch.no_grad():  # Disable gradient computation for teacher
            for model in self.models:
                outputs = model(img, meta)
                model_outputs.append(outputs)

        # Convert list to tensor shape [num_models, batch_size, num_classes]
        model_outputs = torch.stack(model_outputs, dim=0)

        if self.ensemble_method == "mean":
            # Soft-label generation: Averaging logits
            avg_outputs = model_outputs.mean(dim=0)  # Shape [batch_size, num_classes]
        elif self.ensemble_method == "vote":
            # Majority voting: Get the most common prediction
            _, predictions = torch.max(model_outputs, dim=2)  # Shape [num_models, batch_size]
            avg_outputs = predictions.mode(dim=0).values  # Majority vote

        return avg_outputs  # These are the soft labels for KD

teacher_model = TeacherModel(models=[mobilevit_model, pv2_model, swin_model, xception_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[pv2_model, swin_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, pv2_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, swin_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, pv2_model, swin_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, swin_model, xception_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, pv2_model, xception_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[pv2_model, swin_model, xception_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[pv2_model, xception_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[swin_model, xception_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, xception_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, swin_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
teacher_model = TeacherModel(models=[mobilevit_model, pv2_model], ensemble_method="mean")

# Move to the correct device (CPU/GPU)
teacher_model = teacher_model.to(device)

true_labels, pred_labels = test(teacher_model, test_loader, device)

class_names = label_encoder.classes_
report = classification_report(true_labels, pred_labels, digits=4,target_names=class_names)
print("Classification Report:")
print(report)
cm = confusion_matrix(true_labels, pred_labels)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import torch.nn.functional as F

# Load the best model
best_model = EarlyFusionWithDynamicGCN(input_dim_meta=59, num_classes=6).to(device)
best_model.load_state_dict(torch.load("best_student_model_test_loss1.pth"))
best_model.eval()

all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for images, metas, labels in test_loader:
        images, metas, labels = images.to(device), metas.to(device), labels.to(device)
        batch_indices = torch.arange(metas.size(0)).to(device).long()
        outputs = student_model(images, metas, batch_indices)        
        probs = F.softmax(outputs, dim=1)
        preds = probs.argmax(dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
all_probs = np.array(all_probs)

# Compute classification report
class_report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)

# Compute normalized confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds, normalize="true")

# Display classification report
print("\nClassification Report:\n")
print(class_report)

# Display confusion matrix (black and white)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="gray", fmt=".2f", xticklabels=class_names, yticklabels=class_names, cbar=True)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Normalized Confusion Matrix")
plt.show()

# Compute and plot ROC-AUC curve for each class
plt.figure(figsize=(10, 6))

for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve((all_labels == i).astype(int), all_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_name} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], "k--")  # Diagonal line for reference
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-AUC Curve")
plt.legend(loc="lower right")
plt.show()

# Optional: Compute and plot Precision-Recall Curve
plt.figure(figsize=(10, 6))

for i, class_name in enumerate(class_names):
    precision, recall, _ = precision_recall_curve((all_labels == i).astype(int), all_probs[:, i])
    plt.plot(recall, precision, label=f"{class_name}")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()

#### <h1>Majority Voting</h1>

In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

def test_ensemble(models, loader, device):
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            
            # Initialize a list to store individual model predictions
            model_outputs = []
            
            # Get predictions from each model and store
            for model in models:
                outputs = model(imgs, metas)
                model_outputs.append(outputs)
            
            # Stack model outputs along a new axis (axis 0: models)
            model_outputs = torch.stack(model_outputs, dim=0)
            
            # Average the outputs along the new axis (across models)
            avg_outputs = model_outputs.mean(dim=0)  # Averaging logits/probabilities

            # Convert the averaged outputs to predicted labels
            _, predicted = torch.max(avg_outputs, 1)

            # Store the predictions and the true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


# Function to calculate metrics and return mean ± std deviation
def calculate_metrics(true_labels, predictions):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


# Main loop to test the ensemble and calculate metrics with standard deviation
def evaluate_ensemble(models, test_loader, device, seeds):
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Loop through predefined seeds
    for seed in seeds:
        set_seed(seed)  # Set the random seed for reproducibility
        
        # Run the ensemble evaluation
        true_labels, ensemble_preds = test_ensemble(models, test_loader, device)

        # Calculate metrics for this run
        accuracy, precision, recall, f1 = calculate_metrics(true_labels, ensemble_preds)

        # Store metrics
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with 4 models:
models = [mobilevit_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fi``xed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble(models, test_loader, device, seeds)


In [ ]:
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

def test_ensemble(models, loader, device):
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            
            # Initialize a list to store individual model predictions
            model_outputs = []
            
            # Get predictions from each model and store
            for model in models:
                outputs = model(imgs, metas)
                model_outputs.append(outputs)
            
            # Stack model outputs along a new axis (axis 0: models)
            model_outputs = torch.stack(model_outputs, dim=0)
            
            # Average the outputs along the new axis (across models)
            avg_outputs = model_outputs.mean(dim=0)  # Averaging logits/probabilities

            # Convert the averaged outputs to predicted labels
            _, predicted = torch.max(avg_outputs, 1)

            # Store the predictions and the true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


# Function to calculate metrics and return mean ± std deviation
def calculate_metrics(true_labels, predictions):
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


# Main loop to test the ensemble and calculate metrics with standard deviation
def evaluate_ensemble(models, test_loader, device, seeds):
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Loop through predefined seeds
    for seed in seeds:
        set_seed(seed)  # Set the random seed for reproducibility
        
        # Run the ensemble evaluation
        true_labels, ensemble_preds = test_ensemble(models, test_loader, device)

        # Calculate metrics for this run
        accuracy, precision, recall, f1 = calculate_metrics(true_labels, ensemble_preds)

        # Store metrics
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with 4 models:
models = [mobilevit_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fi``xed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble(models, test_loader, device, seeds)


In [ ]:
import torch
import numpy as np
from scipy import stats  # For majority voting

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

def test_ensemble_majority_voting(models, loader, device):
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            
            # Initialize a list to store individual model predictions
            model_preds = []
            
            # Get predictions from each model and store
            for model in models:
                outputs = model(imgs, metas)
                _, predicted = torch.max(outputs, 1)
                model_preds.append(predicted.cpu().numpy())  # Store each model's predictions

            # Majority voting: get the majority class from model_preds
            # `stats.mode()` returns the most frequent element along axis 0 (per sample)
            majority_preds = stats.mode(np.array(model_preds), axis=0)[0]  # Majority vote

            # Store the predictions and the true labels
            all_preds.extend(majority_preds)
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


# Function to calculate metrics and return mean ± std deviation
def calculate_metrics(true_labels, predictions):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


# Main loop to test the ensemble with majority voting and calculate metrics with standard deviation
def evaluate_ensemble_majority_voting(models, test_loader, device, seeds):
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Loop through predefined seeds
    for seed in seeds:
        set_seed(seed)  # Set the random seed for reproducibility
        
        # Run the ensemble evaluation with majority voting
        true_labels, ensemble_preds = test_ensemble_majority_voting(models, test_loader, device)

        # Calculate metrics for this run
        accuracy, precision, recall, f1 = calculate_metrics(true_labels, ensemble_preds)

        # Store metrics
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with 4 models:
models = [mobilevit_model, pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
import torch
import numpy as np
from scipy import stats  # For majority voting

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

def test_ensemble_majority_voting(models, loader, device):
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            
            # Initialize a list to store individual model predictions
            model_preds = []
            
            # Get predictions from each model and store
            for model in models:
                outputs = model(imgs, metas)
                _, predicted = torch.max(outputs, 1)
                model_preds.append(predicted.cpu().numpy())  # Store each model's predictions

            # Majority voting: get the majority class from model_preds
            # `stats.mode()` returns the most frequent element along axis 0 (per sample)
            majority_preds = stats.mode(np.array(model_preds), axis=0)[0]  # Majority vote

            # Store the predictions and the true labels
            all_preds.extend(majority_preds)
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


# Function to calculate metrics and return mean ± std deviation
def calculate_metrics(true_labels, predictions):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


# Main loop to test the ensemble with majority voting and calculate metrics with standard deviation
def evaluate_ensemble_majority_voting(models, test_loader, device, seeds):
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Loop through predefined seeds
    for seed in seeds:
        set_seed(seed)  # Set the random seed for reproducibility
        
        # Run the ensemble evaluation with majority voting
        true_labels, ensemble_preds = test_ensemble_majority_voting(models, test_loader, device)

        # Calculate metrics for this run
        accuracy, precision, recall, f1 = calculate_metrics(true_labels, ensemble_preds)

        # Store metrics
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with 4 models:
models = [pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)


In [ ]:
import torch
import numpy as np
from scipy import stats  # For majority voting

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

def test_ensemble_majority_voting(models, loader, device):
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            
            # Initialize a list to store individual model predictions
            model_preds = []
            
            # Get predictions from each model and store
            for model in models:
                outputs = model(imgs, metas)
                _, predicted = torch.max(outputs, 1)
                model_preds.append(predicted.cpu().numpy())  # Store each model's predictions

            # Majority voting: get the majority class from model_preds
            # `stats.mode()` returns the most frequent element along axis 0 (per sample)
            majority_preds = stats.mode(np.array(model_preds), axis=0)[0]  # Majority vote

            # Store the predictions and the true labels
            all_preds.extend(majority_preds)
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


# Function to calculate metrics and return mean ± std deviation
def calculate_metrics(true_labels, predictions):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


# Main loop to test the ensemble with majority voting and calculate metrics with standard deviation
def evaluate_ensemble_majority_voting(models, test_loader, device, seeds):
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Loop through predefined seeds
    for seed in seeds:
        set_seed(seed)  # Set the random seed for reproducibility
        
        # Run the ensemble evaluation with majority voting
        true_labels, ensemble_preds = test_ensemble_majority_voting(models, test_loader, device)

        # Calculate metrics for this run
        accuracy, precision, recall, f1 = calculate_metrics(true_labels, ensemble_preds)

        # Store metrics
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with 4 models:
models = [mobilevit_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)


In [ ]:
import torch
import numpy as np
from scipy import stats  # For majority voting

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

def test_ensemble_majority_voting(models, loader, device):
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, metas, labels in loader:
            imgs, metas, labels = imgs.to(device), metas.to(device), labels.to(device)
            
            # Initialize a list to store individual model predictions
            model_preds = []
            
            # Get predictions from each model and store
            for model in models:
                outputs = model(imgs, metas)
                _, predicted = torch.max(outputs, 1)
                model_preds.append(predicted.cpu().numpy())  # Store each model's predictions

            # Majority voting: get the majority class from model_preds
            # `stats.mode()` returns the most frequent element along axis 0 (per sample)
            majority_preds = stats.mode(np.array(model_preds), axis=0)[0]  # Majority vote

            # Store the predictions and the true labels
            all_preds.extend(majority_preds)
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds


# Function to calculate metrics and return mean ± std deviation
def calculate_metrics(true_labels, predictions):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    return accuracy, precision, recall, f1


# Main loop to test the ensemble with majority voting and calculate metrics with standard deviation
def evaluate_ensemble_majority_voting(models, test_loader, device, seeds):
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    # Loop through predefined seeds
    for seed in seeds:
        set_seed(seed)  # Set the random seed for reproducibility
        
        # Run the ensemble evaluation with majority voting
        true_labels, ensemble_preds = test_ensemble_majority_voting(models, test_loader, device)

        # Calculate metrics for this run
        accuracy, precision, recall, f1 = calculate_metrics(true_labels, ensemble_preds)

        # Store metrics
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with 4 models:
models = [mobilevit_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)


In [ ]:
models = [mobilevit_model, xception_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [xception_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [xception_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [mobilevit_model, xception_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [mobilevit_model, xception_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [mobilevit_model, pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [mobilevit_model, xception_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [xception_model, pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

In [ ]:
models = [mobilevit_model, xception_model, pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# Assume test_loader is defined (DataLoader for test set)
evaluate_ensemble_majority_voting(models, test_loader, device, seeds)

<h1>Weighted Averaging</h1>

In [ ]:
import torch
import numpy as np
import optuna
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch import nn

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

class WeightedAverageEnsemble(nn.Module):
    def __init__(self, models, weights):
        super(WeightedAverageEnsemble, self).__init__()
        self.models = nn.ModuleList(models)
        self.weights = weights

    def forward(self, x):
        weighted_avg = 0
        for i, model in enumerate(self.models):
            weighted_avg += self.weights[i] * model(x)
        return weighted_avg

def evaluate_ensemble(models, weights, test_loader, device):
    if not isinstance(weights, torch.Tensor):
        weights = torch.tensor(weights, device=device)
    
    # Round to 4 decimal places
    weights = torch.round(weights * 10000) / 10000
    
    # Normalize weights to sum to 1
    weights = weights / weights.sum()

    # Switch models to evaluation mode
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            # Unpack the batch into inputs, meta, and labels
            inputs, meta, labels = batch  # inputs is the image, meta is the metadata

            # Move inputs, meta, and labels to device (GPU/CPU)
            inputs, meta, labels = inputs.to(device), meta.to(device), labels.to(device)

            # Initialize a list to store individual model predictions
            model_outputs = []

            # Get predictions from each model and store
            for model in models:
                # Ensure that each model receives both inputs and meta
                outputs = model(inputs, meta)
                model_outputs.append(outputs)

            # Stack model outputs along a new axis (axis 0: models)
            model_outputs = torch.stack(model_outputs, dim=0)

            # Weighted average of the outputs (according to the weights)
            weighted_avg = torch.zeros_like(model_outputs[0])
            for i, output in enumerate(model_outputs):
                weighted_avg += weights[i] * output

            # Convert the averaged outputs to predicted labels
            _, predicted = torch.max(weighted_avg, 1)

            # Store the predictions and the true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)

    return accuracy, f1, precision, recall, cm

def objective(trial, models, train_loader, val_loader, device):
    weights = [trial.suggest_float(f'weight_{i+1}', 0.0, 1.0) for i in range(len(models))]
    normalized_weights = [round(w, 4) for w in weights]  # Round to 4 decimal places

    # Evaluate the ensemble with the suggested weights
    accuracy, _, _, _, _ = evaluate_ensemble(models, normalized_weights, val_loader, device)
    return accuracy

def evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file):
    best_weights_across_seeds = {}

    # Lists to store metrics for all seeds
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    for seed in seeds:
        # Set the seed for reproducibility
        set_seed(seed)

        # Create a study for hyperparameter optimization
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, models, train_loader, val_loader, device), n_trials=30)

        best_weights = study.best_trial.params
        best_weights_tensor = torch.tensor([round(best_weights[f'weight_{i+1}'], 4) for i in range(len(models))], device=device)
        normalized_best_weights = best_weights_tensor / best_weights_tensor.sum()

        # Save the best weights for this seed
        best_weights_across_seeds[seed] = normalized_best_weights.cpu().numpy().tolist()

        # Print out best weights for this seed
        print(f'Best Weights for Seed {seed}: {normalized_best_weights.cpu().numpy()}')
        print(f'Best Accuracy for Seed {seed}: {study.best_value:.4f}')

        # Evaluate the ensemble with the best weights on the test set
        accuracy, f1, precision, recall, cm = evaluate_ensemble(models, normalized_best_weights, test_loader, device)

        # Collect metrics for later calculation of mean and std deviation
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        # Save the confusion matrix as well for further analysis
        print(f'Confusion Matrix for Seed {seed}:\n{cm}')

    # Save best weights for all seeds
    with open(save_weights_file, 'w') as f:
        json.dump(best_weights_across_seeds, f)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results with mean and standard deviation
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with models:
models = [mobilevit_model, pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')

In [ ]:
import torch

torch.cuda.empty_cache()  # Clear unused GPU memory cache


In [ ]:
import torch
import numpy as np
import optuna
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch import nn

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

class WeightedAverageEnsemble(nn.Module):
    def __init__(self, models, weights):
        super(WeightedAverageEnsemble, self).__init__()
        self.models = nn.ModuleList(models)
        self.weights = weights

    def forward(self, x):
        weighted_avg = 0
        for i, model in enumerate(self.models):
            weighted_avg += self.weights[i] * model(x)
        return weighted_avg

def evaluate_ensemble(models, weights, test_loader, device):
    if not isinstance(weights, torch.Tensor):
        weights = torch.tensor(weights, device=device)
    
    # Round to 4 decimal places
    weights = torch.round(weights * 10000) / 10000
    
    # Normalize weights to sum to 1
    weights = weights / weights.sum()

    # Switch models to evaluation mode
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            # Unpack the batch into inputs, meta, and labels
            inputs, meta, labels = batch  # inputs is the image, meta is the metadata

            # Move inputs, meta, and labels to device (GPU/CPU)
            inputs, meta, labels = inputs.to(device), meta.to(device), labels.to(device)

            # Initialize a list to store individual model predictions
            model_outputs = []

            # Get predictions from each model and store
            for model in models:
                # Ensure that each model receives both inputs and meta
                outputs = model(inputs, meta)
                model_outputs.append(outputs)

            # Stack model outputs along a new axis (axis 0: models)
            model_outputs = torch.stack(model_outputs, dim=0)

            # Weighted average of the outputs (according to the weights)
            weighted_avg = torch.zeros_like(model_outputs[0])
            for i, output in enumerate(model_outputs):
                weighted_avg += weights[i] * output

            # Convert the averaged outputs to predicted labels
            _, predicted = torch.max(weighted_avg, 1)

            # Store the predictions and the true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)

    return accuracy, f1, precision, recall, cm

def objective(trial, models, train_loader, val_loader, device):
    weights = [trial.suggest_float(f'weight_{i+1}', 0.0, 1.0) for i in range(len(models))]
    normalized_weights = [round(w, 4) for w in weights]  # Round to 4 decimal places

    # Evaluate the ensemble with the suggested weights
    accuracy, _, _, _, _ = evaluate_ensemble(models, normalized_weights, val_loader, device)
    return accuracy

def evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file):
    best_weights_across_seeds = {}

    # Lists to store metrics for all seeds
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    for seed in seeds:
        # Set the seed for reproducibility
        set_seed(seed)

        # Create a study for hyperparameter optimization
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, models, train_loader, val_loader, device), n_trials=30)

        best_weights = study.best_trial.params
        best_weights_tensor = torch.tensor([round(best_weights[f'weight_{i+1}'], 4) for i in range(len(models))], device=device)
        normalized_best_weights = best_weights_tensor / best_weights_tensor.sum()

        # Save the best weights for this seed
        best_weights_across_seeds[seed] = normalized_best_weights.cpu().numpy().tolist()

        # Print out best weights for this seed
        print(f'Best Weights for Seed {seed}: {normalized_best_weights.cpu().numpy()}')
        print(f'Best Accuracy for Seed {seed}: {study.best_value:.4f}')

        # Evaluate the ensemble with the best weights on the test set
        accuracy, f1, precision, recall, cm = evaluate_ensemble(models, normalized_best_weights, test_loader, device)

        # Collect metrics for later calculation of mean and std deviation
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        # Save the confusion matrix as well for further analysis
        print(f'Confusion Matrix for Seed {seed}:\n{cm}')

    # Save best weights for all seeds
    with open(save_weights_file, 'w') as f:
        json.dump(best_weights_across_seeds, f)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results with mean and standard deviation
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with models:
models = [pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights1.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')

torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
import torch

torch.cuda.empty_cache()  # Clear unused GPU memory cache


In [ ]:
import torch
import numpy as np
import optuna
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch import nn

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

class WeightedAverageEnsemble(nn.Module):
    def __init__(self, models, weights):
        super(WeightedAverageEnsemble, self).__init__()
        self.models = nn.ModuleList(models)
        self.weights = weights

    def forward(self, x):
        weighted_avg = 0
        for i, model in enumerate(self.models):
            weighted_avg += self.weights[i] * model(x)
        return weighted_avg

def evaluate_ensemble(models, weights, test_loader, device):
    if not isinstance(weights, torch.Tensor):
        weights = torch.tensor(weights, device=device)
    
    # Round to 4 decimal places
    weights = torch.round(weights * 10000) / 10000
    
    # Normalize weights to sum to 1
    weights = weights / weights.sum()

    # Switch models to evaluation mode
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            # Unpack the batch into inputs, meta, and labels
            inputs, meta, labels = batch  # inputs is the image, meta is the metadata

            # Move inputs, meta, and labels to device (GPU/CPU)
            inputs, meta, labels = inputs.to(device), meta.to(device), labels.to(device)

            # Initialize a list to store individual model predictions
            model_outputs = []

            # Get predictions from each model and store
            for model in models:
                # Ensure that each model receives both inputs and meta
                outputs = model(inputs, meta)
                model_outputs.append(outputs)

            # Stack model outputs along a new axis (axis 0: models)
            model_outputs = torch.stack(model_outputs, dim=0)

            # Weighted average of the outputs (according to the weights)
            weighted_avg = torch.zeros_like(model_outputs[0])
            for i, output in enumerate(model_outputs):
                weighted_avg += weights[i] * output

            # Convert the averaged outputs to predicted labels
            _, predicted = torch.max(weighted_avg, 1)

            # Store the predictions and the true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)

    return accuracy, f1, precision, recall, cm

def objective(trial, models, train_loader, val_loader, device):
    weights = [trial.suggest_float(f'weight_{i+1}', 0.0, 1.0) for i in range(len(models))]
    normalized_weights = [round(w, 4) for w in weights]  # Round to 4 decimal places

    # Evaluate the ensemble with the suggested weights
    accuracy, _, _, _, _ = evaluate_ensemble(models, normalized_weights, val_loader, device)
    return accuracy

def evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file):
    best_weights_across_seeds = {}

    # Lists to store metrics for all seeds
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    for seed in seeds:
        # Set the seed for reproducibility
        set_seed(seed)

        # Create a study for hyperparameter optimization
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, models, train_loader, val_loader, device), n_trials=30)

        best_weights = study.best_trial.params
        best_weights_tensor = torch.tensor([round(best_weights[f'weight_{i+1}'], 4) for i in range(len(models))], device=device)
        normalized_best_weights = best_weights_tensor / best_weights_tensor.sum()

        # Save the best weights for this seed
        best_weights_across_seeds[seed] = normalized_best_weights.cpu().numpy().tolist()

        # Print out best weights for this seed
        print(f'Best Weights for Seed {seed}: {normalized_best_weights.cpu().numpy()}')
        print(f'Best Accuracy for Seed {seed}: {study.best_value:.4f}')

        # Evaluate the ensemble with the best weights on the test set
        accuracy, f1, precision, recall, cm = evaluate_ensemble(models, normalized_best_weights, test_loader, device)

        # Collect metrics for later calculation of mean and std deviation
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        # Save the confusion matrix as well for further analysis
        print(f'Confusion Matrix for Seed {seed}:\n{cm}')

    # Save best weights for all seeds
    with open(save_weights_file, 'w') as f:
        json.dump(best_weights_across_seeds, f)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results with mean and standard deviation
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with models:
models = [mobilevit_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights2.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
import torch
import numpy as np
import optuna
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch import nn

# Function to set random seeds for reproducibility
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.cuda.manual_seed_all(seed)  # If using CUDA

class WeightedAverageEnsemble(nn.Module):
    def __init__(self, models, weights):
        super(WeightedAverageEnsemble, self).__init__()
        self.models = nn.ModuleList(models)
        self.weights = weights

    def forward(self, x):
        weighted_avg = 0
        for i, model in enumerate(self.models):
            weighted_avg += self.weights[i] * model(x)
        return weighted_avg

def evaluate_ensemble(models, weights, test_loader, device):
    if not isinstance(weights, torch.Tensor):
        weights = torch.tensor(weights, device=device)
    
    # Round to 4 decimal places
    weights = torch.round(weights * 10000) / 10000
    
    # Normalize weights to sum to 1
    weights = weights / weights.sum()

    # Switch models to evaluation mode
    for model in models:
        model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            # Unpack the batch into inputs, meta, and labels
            inputs, meta, labels = batch  # inputs is the image, meta is the metadata

            # Move inputs, meta, and labels to device (GPU/CPU)
            inputs, meta, labels = inputs.to(device), meta.to(device), labels.to(device)

            # Initialize a list to store individual model predictions
            model_outputs = []

            # Get predictions from each model and store
            for model in models:
                # Ensure that each model receives both inputs and meta
                outputs = model(inputs, meta)
                model_outputs.append(outputs)

            # Stack model outputs along a new axis (axis 0: models)
            model_outputs = torch.stack(model_outputs, dim=0)

            # Weighted average of the outputs (according to the weights)
            weighted_avg = torch.zeros_like(model_outputs[0])
            for i, output in enumerate(model_outputs):
                weighted_avg += weights[i] * output

            # Convert the averaged outputs to predicted labels
            _, predicted = torch.max(weighted_avg, 1)

            # Store the predictions and the true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    cm = confusion_matrix(all_labels, all_preds)

    return accuracy, f1, precision, recall, cm

def objective(trial, models, train_loader, val_loader, device):
    weights = [trial.suggest_float(f'weight_{i+1}', 0.0, 1.0) for i in range(len(models))]
    normalized_weights = [round(w, 4) for w in weights]  # Round to 4 decimal places

    # Evaluate the ensemble with the suggested weights
    accuracy, _, _, _, _ = evaluate_ensemble(models, normalized_weights, val_loader, device)
    return accuracy

def evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file):
    best_weights_across_seeds = {}

    # Lists to store metrics for all seeds
    accuracies, precisions, recalls, f1_scores = [], [], [], []

    for seed in seeds:
        # Set the seed for reproducibility
        set_seed(seed)

        # Create a study for hyperparameter optimization
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, models, train_loader, val_loader, device), n_trials=30)

        best_weights = study.best_trial.params
        best_weights_tensor = torch.tensor([round(best_weights[f'weight_{i+1}'], 4) for i in range(len(models))], device=device)
        normalized_best_weights = best_weights_tensor / best_weights_tensor.sum()

        # Save the best weights for this seed
        best_weights_across_seeds[seed] = normalized_best_weights.cpu().numpy().tolist()

        # Print out best weights for this seed
        print(f'Best Weights for Seed {seed}: {normalized_best_weights.cpu().numpy()}')
        print(f'Best Accuracy for Seed {seed}: {study.best_value:.4f}')

        # Evaluate the ensemble with the best weights on the test set
        accuracy, f1, precision, recall, cm = evaluate_ensemble(models, normalized_best_weights, test_loader, device)

        # Collect metrics for later calculation of mean and std deviation
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        # Save the confusion matrix as well for further analysis
        print(f'Confusion Matrix for Seed {seed}:\n{cm}')

    # Save best weights for all seeds
    with open(save_weights_file, 'w') as f:
        json.dump(best_weights_across_seeds, f)

    # Calculate mean and standard deviation for each metric
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)

    mean_precision = np.mean(precisions)
    std_precision = np.std(precisions)

    mean_recall = np.mean(recalls)
    std_recall = np.std(recalls)

    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)

    # Print out results with mean and standard deviation
    print(f"Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"Precision: {mean_precision:.4f} ± {std_precision:.4f}")
    print(f"Recall: {mean_recall:.4f} ± {std_recall:.4f}")
    print(f"F1 Score: {mean_f1:.4f} ± {std_f1:.4f}")


# Example usage with models:
models = [mobilevit_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
models = [mobilevit_model, xception_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
models = [xception_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
models = [xception_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
models = [mobilevit_model, xception_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
models = [mobilevit_model, xception_model, pv2_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
models = [pv2_model, xception_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache

In [ ]:
models = [mobilevit_model, xception_model, pv2_model, swin_model]  # List of models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fixed random seeds to use
seeds = [42, 123, 569]

# File path to save the best weights
save_weights_file = 'best_weights3.json'

# Assume test_loader, train_loader, and val_loader are defined (DataLoader for test, train, and validation sets)
evaluate_ensemble_with_optuna(models, train_loader, val_loader, test_loader, device, seeds, save_weights_file)

# Example of loading weights and using them for prediction
with open(save_weights_file, 'r') as f:
    best_weights = json.load(f)

# Example: print the best weights for seed 42
print(f'Best Weights for Seed 42: {best_weights["42"]}')
torch.cuda.empty_cache()  # Clear unused GPU memory cache